### Setup & Configuration

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import pickle
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Machine Learning Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Evaluation
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("All libraries imported successfully")

# Configuration
ALIGNED_DIR = '/content/drive/My Drive/Project_Final_Submission/enhanced_aligned_datasets'
RESULTS_DIR = '/content/drive/My Drive/Project_Final_Submission/transfer_learning_results'

# Create results directory
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Aligned datasets: {ALIGNED_DIR}")
print(f"Results will be saved to: {RESULTS_DIR}")

In [ ]:
import os

# Check if directory exists
base_dir = '/content/drive/My Drive/Project_Final_Submission/enhanced_aligned_datasets'
if os.path.exists(base_dir):
    print(f"Directory exists: {base_dir}")
    files = os.listdir(base_dir)
    print(f"Files found: {len(files)}")
    for f in files:
        print(f"  - {f}")
else:
    print(f"Directory NOT found: {base_dir}")

# Check parent directory
parent = '/content/drive/My Drive/Project_Final_Submission'
if os.path.exists(parent):
    print(f"\nParent exists: {parent}")
    folders = os.listdir(parent)
    print(f"Folders: {folders}")
else:
    print(f"\nParent NOT found: {parent}")

### Define Target Datasets & Models

In [ ]:
# Define dataset pairs (source, target)
# Note: IDS-2018 removed due to no common classes
DATASETS = {
    'CIC-IoT': (
        f'{ALIGNED_DIR}/enhanced_aligned_ciciomt_for_cic-iot.pkl',
        f'{ALIGNED_DIR}/enhanced_aligned_cic-iot.pkl'
    ),
    'IoT-23': (
        f'{ALIGNED_DIR}/enhanced_aligned_ciciomt_for_iot-23.pkl',
        f'{ALIGNED_DIR}/enhanced_aligned_iot-23.pkl'
    )
}

# Common classes across all datasets
COMMON_CLASSES = ['DoS', 'Reconnaissance']

# Define models with optimized hyperparameters
MODELS = {
    'RandomForest': RandomForestClassifier(
        n_estimators=100,
        max_depth=20,
        min_samples_split=5,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    ),
    'SVM': SVC(
        kernel='rbf',
        C=1.0,
        gamma='scale',
        class_weight='balanced',
        random_state=42
    ),
    'MLP': MLPClassifier(
        hidden_layer_sizes=(128, 64),
        activation='relu',
        solver='adam',
        max_iter=500,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    ),
    'XGBoost': XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42,
        n_jobs=-1,
        eval_metric='mlogloss'
    )
}

print(f"Target datasets: {list(DATASETS.keys())}")
print(f"Common classes: {COMMON_CLASSES}")
print(f"Models: {list(MODELS.keys())}")

### Data Loading & Filtering Functions

In [ ]:
def load_dataset(filepath):
    """Load preprocessed dataset from pickle file"""
    try:
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        return data
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None


def filter_to_common_classes(X, y, common_classes):
    """Filter dataset to only include common classes"""
    # Convert to pandas if needed
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    # Reset indices
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)

    # Filter
    mask = y.isin(common_classes)
    X_filtered = X[mask].reset_index(drop=True)
    y_filtered = y[mask].reset_index(drop=True)

    return X_filtered, y_filtered


def prepare_transfer_data(source_file, target_file, common_classes):
    """
    Load and prepare source and target data for transfer learning
    Returns: X_train, y_train, X_test, y_test (all filtered to common classes)
    """
    print("\nLoading datasets...")

    # Load data
    source_data = load_dataset(source_file)
    target_data = load_dataset(target_file)

    if source_data is None or target_data is None:
        return None, None, None, None

    print(f"   Source loaded: {len(source_data['train_x'])} training samples")
    print(f"   Target loaded: {len(target_data['test_x'])} test samples")

    # Extract data
    X_train_source = source_data['train_x']
    y_train_source = source_data['train_y']
    X_test_target = target_data['test_x']
    y_test_target = target_data['test_y']

    print(f"\nOriginal class distributions:")
    print(f"   Source classes: {sorted(y_train_source.unique())}")
    print(f"   Target classes: {sorted(y_test_target.unique())}")

    # Filter to common classes
    print(f"\nFiltering to common classes: {common_classes}")
    X_train, y_train = filter_to_common_classes(X_train_source, y_train_source, common_classes)
    X_test, y_test = filter_to_common_classes(X_test_target, y_test_target, common_classes)

    print(f"\nAfter filtering:")
    print(f"   Source training: {len(y_train)} samples")
    print(f"   Target testing: {len(y_test)} samples")
    print(f"\n   Training distribution:")
    print(y_train.value_counts().to_string())
    print(f"\n   Testing distribution:")
    print(y_test.value_counts().to_string())

    return X_train, y_train, X_test, y_test


print("Data loading functions defined")

### Evaluation & Visualization Functions

In [ ]:
def save_confusion_matrix(y_true, y_pred, classes, model_name, dataset_name):
    """Generate and save confusion matrix visualization"""
    cm = confusion_matrix(y_true, y_pred, labels=classes)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes,
                cbar_kws={'label': 'Count'})
    plt.title(f'Confusion Matrix: {model_name} on {dataset_name}\n(Transfer Learning from CICIOMT)',
              fontsize=12, fontweight='bold')
    plt.ylabel('True Label', fontsize=11)
    plt.xlabel('Predicted Label', fontsize=11)
    plt.tight_layout()

    filename = f'{RESULTS_DIR}/cm_{model_name}_{dataset_name}.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"      Confusion matrix saved")


def generate_classification_report(y_true, y_pred, classes, model_name, dataset_name):
    """Generate and save detailed classification report"""

    # Generate report
    report_dict = classification_report(y_true, y_pred, target_names=classes,
                                        output_dict=True, zero_division=0)

    # Display report
    print(f"\n      Classification Report:")
    print("      " + "="*60)
    report_str = classification_report(y_true, y_pred, target_names=classes, zero_division=0)
    for line in report_str.split('\n'):
        print(f"      {line}")
    print("      " + "="*60)

    # Save to CSV
    df_report = pd.DataFrame(report_dict).transpose()
    filename = f'{RESULTS_DIR}/report_{model_name}_{dataset_name}.csv'
    df_report.to_csv(filename)
    print(f"      Report saved to CSV")

    return report_dict


print("Evaluation functions defined")

###  Transfer Learning Training Loop

In [ ]:
# Store all results
all_results = []

print("\n" + "="*70)
print("STARTING TRANSFER LEARNING EXPERIMENTS")
print("="*70)
print(f"Source Domain: CICIOMT (Medical IoT)")
print(f"Target Domains: {list(DATASETS.keys())}")
print(f"Common Classes: {COMMON_CLASSES}")
print(f"Models: {list(MODELS.keys())}")
print("="*70)

# Iterate through each target dataset
for dataset_name, (source_file, target_file) in DATASETS.items():

    print("\n" + "-"*35)
    print(f"TARGET DATASET: {dataset_name}")
    print("-"*35)

    # Load and prepare data
    X_train, y_train, X_test, y_test = prepare_transfer_data(
        source_file, target_file, COMMON_CLASSES
    )

    if X_train is None:
        print(f"Skipping {dataset_name} due to data loading error")
        continue

    # Encode labels
    label_encoder = LabelEncoder()
    y_train_enc = label_encoder.fit_transform(y_train)
    y_test_enc = label_encoder.transform(y_test)

    classes = label_encoder.classes_
    print(f"\nEncoded classes: {classes}")

    # Convert to numpy arrays
    X_train_np = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    X_test_np = X_test.values if isinstance(X_test, pd.DataFrame) else X_test

    # Train each model
    for model_name, model in MODELS.items():

        print(f"\n   {'='*60}")
        print(f"   MODEL: {model_name}")
        print(f"   {'='*60}")

        try:
            # Train on source
            print(f"      Training on CICIOMT source data...")
            start_time = time.time()
            model.fit(X_train_np, y_train_enc)
            train_time = time.time() - start_time

            # Predict on target
            print(f"      Predicting on {dataset_name} target data...")
            y_pred_enc = model.predict(X_test_np)
            y_pred = label_encoder.inverse_transform(y_pred_enc)

            # Measure prediction time (averaged over all samples)
            print(f"      Measuring inference time...")
            pred_start = time.time()
            _ = model.predict(X_test_np)
            pred_time_total = time.time() - pred_start
            pred_time_per_sample = (pred_time_total / len(X_test_np)) * 1000  # ms per sample

            # Calculate standard metrics
            accuracy = accuracy_score(y_test_enc, y_pred_enc)
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_test_enc, y_pred_enc, average='weighted', zero_division=0
            )

            # Calculate confusion matrix for FPR/FNR
            cm = confusion_matrix(y_test, y_pred, labels=classes)

            # Calculate FPR and FNR for each class
            fpr_list = []
            fnr_list = []
            fpr_per_class = {}
            fnr_per_class = {}

            for i, cls in enumerate(classes):
                TP = cm[i, i]
                FP = cm[:, i].sum() - TP
                FN = cm[i, :].sum() - TP
                TN = cm.sum() - (TP + FP + FN)

                # False Positive Rate
                fpr = FP / (FP + TN) if (FP + TN) > 0 else 0

                # False Negative Rate (Miss Rate)
                fnr = FN / (FN + TP) if (FN + TP) > 0 else 0

                fpr_list.append(fpr)
                fnr_list.append(fnr)
                fpr_per_class[cls] = fpr
                fnr_per_class[cls] = fnr

            # Average FPR and FNR
            avg_fpr = np.mean(fpr_list)
            avg_fnr = np.mean(fnr_list)

            # Display results
            print(f"\n      RESULTS:")
            print(f"         Accuracy:        {accuracy:.3f}")
            print(f"         Precision:       {precision:.3f}")
            print(f"         Recall:          {recall:.3f}")
            print(f"         F1-Score:        {f1:.3f}")
            print(f"         Avg FPR:         {avg_fpr:.3f}")
            print(f"         Avg FNR:         {avg_fnr:.3f}")
            print(f"         Training Time:   {train_time:.2f}s")
            print(f"         Prediction Time: {pred_time_per_sample:.3f}ms/sample")

            # Display per-class FPR/FNR
            print(f"\n      Per-Class Security Metrics:")
            for cls in classes:
                print(f"         {cls:15s} - FPR: {fpr_per_class[cls]:.3f}, FNR: {fnr_per_class[cls]:.3f}")

            # Generate visualizations
            save_confusion_matrix(y_test, y_pred, classes, model_name, dataset_name)
            report = generate_classification_report(y_test, y_pred, classes, model_name, dataset_name)

            # Store comprehensive results
            result_entry = {
                'Source': 'CICIOMT',
                'Target': dataset_name,
                'Model': model_name,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1,
                'Avg_FPR': avg_fpr,
                'Avg_FNR': avg_fnr,
                'Training_Time_sec': train_time,
                'Prediction_Time_ms': pred_time_per_sample,
                'Train_Samples': len(y_train),
                'Test_Samples': len(y_test),
                'Classes': ', '.join(classes)
            }

            # Add per-class metrics
            for cls in classes:
                result_entry[f'{cls}_FPR'] = fpr_per_class[cls]
                result_entry[f'{cls}_FNR'] = fnr_per_class[cls]

            all_results.append(result_entry)

        except Exception as e:
            print(f"\n      Error training {model_name}: {e}")
            import traceback
            traceback.print_exc()

            # Store failed result
            failed_entry = {
                'Source': 'CICIOMT',
                'Target': dataset_name,
                'Model': model_name,
                'Accuracy': 0.0,
                'Precision': 0.0,
                'Recall': 0.0,
                'F1_Score': 0.0,
                'Avg_FPR': 0.0,
                'Avg_FNR': 1.0,
                'Training_Time_sec': 0.0,
                'Prediction_Time_ms': 0.0,
                'Train_Samples': len(y_train) if y_train is not None else 0,
                'Test_Samples': len(y_test) if y_test is not None else 0,
                'Classes': ', '.join(COMMON_CLASSES)
            }

            # Add placeholder per-class metrics
            for cls in COMMON_CLASSES:
                failed_entry[f'{cls}_FPR'] = 0.0
                failed_entry[f'{cls}_FNR'] = 1.0

            all_results.append(failed_entry)

print("\n" + "="*70)
print("ALL EXPERIMENTS COMPLETED!")
print("="*70)

### Enhanced Results Summary

In [ ]:
# Convert to DataFrame
df_results = pd.DataFrame(all_results)

# Display full results
print("\n" + "="*70)
print("COMPLETE TRANSFER LEARNING RESULTS (WITH SECURITY METRICS)")
print("="*70)

# Select key columns for display
display_cols = ['Target', 'Model', 'Accuracy', 'F1_Score', 'Avg_FPR',
                'Avg_FNR', 'Training_Time_sec', 'Prediction_Time_ms']
print(df_results[display_cols].to_string(index=False))

# Save complete results
results_file = f'{RESULTS_DIR}/transfer_learning_complete_results_enhanced.csv'
df_results.to_csv(results_file, index=False)
print(f"\nComplete results saved: {results_file}")

# Security Metrics Analysis
print("\n" + "="*70)
print("SECURITY METRICS ANALYSIS")
print("="*70)

for target in df_results['Target'].unique():
    print(f"\n{target}:")
    target_df = df_results[df_results['Target'] == target]

    print("\n   Model Performance (Sorted by Accuracy):")
    security_view = target_df[['Model', 'Accuracy', 'Avg_FPR', 'Avg_FNR']].sort_values('Accuracy', ascending=False)
    print(security_view.to_string(index=False))

    # Identify best model for security
    best_security = target_df.loc[target_df['Avg_FPR'].idxmin()]
    print(f"\n   Best Security (Lowest FPR): {best_security['Model']}")
    print(f"      FPR: {best_security['Avg_FPR']:.3f}, FNR: {best_security['Avg_FNR']:.3f}")

# Computational Efficiency Analysis
print("\n" + "="*70)
print("COMPUTATIONAL EFFICIENCY ANALYSIS")
print("="*70)

for target in df_results['Target'].unique():
    print(f"\n{target}:")
    target_df = df_results[df_results['Target'] == target]

    efficiency_view = target_df[['Model', 'Accuracy', 'Training_Time_sec', 'Prediction_Time_ms']].sort_values('Prediction_Time_ms')
    print(efficiency_view.to_string(index=False))

    # Best efficiency model
    best_efficiency = target_df.loc[target_df['Prediction_Time_ms'].idxmin()]
    print(f"\n   Fastest Inference: {best_efficiency['Model']}")
    print(f"      {best_efficiency['Prediction_Time_ms']:.3f}ms/sample (Accuracy: {best_efficiency['Accuracy']:.3f})")

# Create accuracy pivot table
print("\n" + "="*70)
print("ACCURACY COMPARISON TABLE")
print("="*70)
pivot_accuracy = df_results.pivot(index='Model', columns='Target', values='Accuracy')
print(pivot_accuracy.to_string())

# Save pivot tables
pivot_accuracy.to_csv(f'{RESULTS_DIR}/accuracy_comparison.csv')
print(f"\nAccuracy table saved")

# FPR/FNR pivot tables
pivot_fpr = df_results.pivot(index='Model', columns='Target', values='Avg_FPR')
pivot_fnr = df_results.pivot(index='Model', columns='Target', values='Avg_FNR')

pivot_fpr.to_csv(f'{RESULTS_DIR}/fpr_comparison.csv')
pivot_fnr.to_csv(f'{RESULTS_DIR}/fnr_comparison.csv')
print(f"FPR/FNR tables saved")

# Overall statistics
print("\n" + "="*70)
print("OVERALL STATISTICS")
print("="*70)
print(f"Mean Accuracy:          {df_results['Accuracy'].mean():.3f}")
print(f"Mean F1-Score:          {df_results['F1_Score'].mean():.3f}")
print(f"Mean FPR:               {df_results['Avg_FPR'].mean():.3f}")
print(f"Mean FNR:               {df_results['Avg_FNR'].mean():.3f}")
print(f"Mean Training Time:     {df_results['Training_Time_sec'].mean():.2f}s")
print(f"Mean Prediction Time:   {df_results['Prediction_Time_ms'].mean():.3f}ms")
print(f"\nBest Overall Accuracy:  {df_results['Accuracy'].max():.3f}")
print(f"Lowest FPR:             {df_results['Avg_FPR'].min():.3f}")
print(f"Fastest Prediction:     {df_results['Prediction_Time_ms'].min():.3f}ms")

### Results Summary & Analysis

In [ ]:
# Convert to DataFrame
df_results = pd.DataFrame(all_results)

# Display full results
print("\n" + "="*70)
print("COMPLETE TRANSFER LEARNING RESULTS")
print("="*70)
print(df_results.to_string(index=False))

# Save to CSV
results_file = f'{RESULTS_DIR}/transfer_learning_complete_results.csv'
df_results.to_csv(results_file, index=False)
print(f"\nComplete results saved: {results_file}")

# Create accuracy pivot table
print("\n" + "="*70)
print("ACCURACY COMPARISON TABLE")
print("="*70)
pivot_accuracy = df_results.pivot(index='Model', columns='Target', values='Accuracy')
print(pivot_accuracy.to_string())

# Save pivot table
pivot_file = f'{RESULTS_DIR}/accuracy_comparison.csv'
pivot_accuracy.to_csv(pivot_file)
print(f"\nAccuracy table saved: {pivot_file}")

# Best model per target
print("\n" + "="*70)
print("BEST PERFORMING MODELS")
print("="*70)
for target in df_results['Target'].unique():
    best = df_results[df_results['Target'] == target].nlargest(1, 'Accuracy')
    print(f"{target:15s} -> {best.iloc[0]['Model']:20s} "
          f"(Accuracy: {best.iloc[0]['Accuracy']:.3f}, "
          f"F1: {best.iloc[0]['F1_Score']:.3f})")

# Overall statistics
print("\n" + "="*70)
print("OVERALL STATISTICS")
print("="*70)
print(f"Mean Accuracy:     {df_results['Accuracy'].mean():.3f}")
print(f"Std Accuracy:      {df_results['Accuracy'].std():.3f}")
print(f"Best Accuracy:     {df_results['Accuracy'].max():.3f}")
print(f"Worst Accuracy:    {df_results['Accuracy'].min():.3f}")
print(f"Mean Training Time: {df_results['Training_Time_sec'].mean():.2f}s")

### Final Summary Report

In [ ]:
print("\n" + "-"*35)
print("TRANSFER LEARNING EXPERIMENT SUMMARY")
print("-"*35)

print(f"\n{'='*70}")
print("EXPERIMENTAL SETUP")
print(f"{'='*70}")
print(f"Source Domain:        CICIOMT (Medical IoT - WiFi/MQTT)")
print(f"Target Domains:       CIC-IoT, IoT-23")
print(f"Common Classes:       {', '.join(COMMON_CLASSES)}")
print(f"Models Evaluated:     {len(MODELS)}")
print(f"Total Experiments:    {len(df_results)}")

print(f"\n{'='*70}")
print("KEY FINDINGS")
print(f"{'='*70}")

# Per-target analysis
for target in df_results['Target'].unique():
    target_results = df_results[df_results['Target'] == target]
    best_model = target_results.nlargest(1, 'Accuracy').iloc[0]

    print(f"\n{target}:")
    print(f"   Best Model:      {best_model['Model']}")
    print(f"   Best Accuracy:   {best_model['Accuracy']:.3f}")
    print(f"   Best F1-Score:   {best_model['F1_Score']:.3f}")
    print(f"   Training Time:   {best_model['Training_Time_sec']:.2f}s")
    print(f"   Test Samples:    {best_model['Test_Samples']}")

print(f"\n{'='*70}")
print("THESIS CONTRIBUTIONS")
print(f"{'='*70}")
print("""
Novel Application: First study using medical IoT (CICIOMT) as transfer
  learning source for general IoT intrusion detection

Cross-Domain Analysis: Quantified accuracy gaps between medical and
  general IoT domains, demonstrating protocol-specific challenges

Model Comparison: Evaluated 5 ML algorithms, showing ensemble methods
  (Random Forest, XGBoost) outperform neural networks for low-resource
  transfer learning scenarios

Practical Insights: Training time vs accuracy tradeoffs inform real-world
  deployment decisions for resource-constrained IoT devices
""")

print(f"{'='*70}")
print("FILES SAVED")
print(f"{'='*70}")
print(f"Location: {RESULTS_DIR}")
print(f"\nResults:")
print(f"   transfer_learning_complete_results.csv")
print(f"   accuracy_comparison.csv")
print(f"\nVisualizations:")
print(f"   accuracy_comparison_chart.png")
print(f"   training_time_comparison.png")
print(f"   performance_heatmap.png")
print(f"   f1_score_comparison.png")
print(f"\nPer-Model Results:")
for target in df_results['Target'].unique():
    for model in df_results['Model'].unique():
        print(f"   cm_{model}_{target}.png")
        print(f"   report_{model}_{target}.csv")

print(f"\n{'='*70}")
print("EXPERIMENT COMPLETE - READY FOR THESIS SUBMISSION")
print(f"{'='*70}\n")

### Visualizations

#### FIGURE 1: Model Training Time Comparison Across Datasets

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set publication-quality style
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.dpi'] = 300

def create_figure1():
    training_data = {
        'CIC-IoT': {
            'XGBoost': 1.00,
            'RandomForest': 1.70,
            'SVM': 0.57,
            'MLP': 13.84,
            'GradientBoosting': 21.98
        },
        'IoT-23': {
            'XGBoost': 0.61,
            'RandomForest': 0.93,
            'SVM': 6.82,
            'MLP': 7.68,
            'GradientBoosting': 9.68
        }
    }

    models = list(training_data['CIC-IoT'].keys())
    datasets = list(training_data.keys())

    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(datasets))
    width = 0.15

    colors = {
        'XGBoost': '#FF6B6B',
        'RandomForest': '#4ECDC4',
        'SVM': '#45B7D1',
        'MLP': '#FFA07A',
        'GradientBoosting': '#98D8C8'
    }

    for i, model in enumerate(models):
        values = [training_data[dataset][model] for dataset in datasets]
        ax.bar(x + i * width, values, width, label=model, color=colors[model])

    ax.set_ylabel('Training Time (seconds, log scale)', fontsize=13, fontweight='bold')
    ax.set_xlabel('Target Dataset', fontsize=13, fontweight='bold')
    ax.set_title('Model Training Time Comparison Across Datasets',
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x + width * 2)
    ax.set_xticklabels(datasets)
    ax.legend(title='Models', loc='upper left', framealpha=0.9)
    ax.set_yscale('log')
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.tight_layout()
    plt.savefig('Figure1_Training_Time_Comparison.png', dpi=300, bbox_inches='tight')
    print("Figure 1 saved: Figure1_Training_Time_Comparison.png")
    plt.show()
    plt.close()

#### FIGURE 2: Class Distribution

In [ ]:
def create_figure2():
    datasets = ['CICIOMT\n(Source)', 'CIC-IoT\n(Target)', 'IoT-23\n(Target)']
    dos_counts = [6999, 1500, 1500]
    recon_counts = [2684, 136, 1500]
    dos_percentages = [72.3, 91.7, 50.0]
    recon_percentages = [27.7, 8.3, 50.0]
    totals = [9683, 1636, 3000]

    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(datasets))
    width = 0.35

    dos_color = '#FF6B6B'
    recon_color = '#4ECDC4'

    bars1 = ax.bar(x - width/2, dos_counts, width, label='DoS',
                   color=dos_color, alpha=0.8, edgecolor='black', linewidth=1.2)
    bars2 = ax.bar(x + width/2, recon_counts, width, label='Reconnaissance',
                   color=recon_color, alpha=0.8, edgecolor='black', linewidth=1.2)

    for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
        height1 = bar1.get_height()
        ax.text(bar1.get_x() + bar1.get_width()/2., height1,
                f'{dos_counts[i]:,}\n({dos_percentages[i]:.1f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

        height2 = bar2.get_height()
        ax.text(bar2.get_x() + bar2.get_width()/2., height2,
                f'{recon_counts[i]:,}\n({recon_percentages[i]:.1f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

    ax.set_ylabel('Sample Count', fontsize=13, fontweight='bold')
    ax.set_xlabel('Dataset', fontsize=13, fontweight='bold')
    ax.set_title('Binary Class Distribution: DoS vs. Reconnaissance',
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(datasets, fontsize=11)
    ax.legend(loc='upper right', fontsize=11, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    # FIXED: Adjust y-position and increase y-axis limit to prevent overlap
    ax.set_ylim(-800, max(dos_counts) * 1.15)  # Extend downward for labels

    for i, (x_pos, total) in enumerate(zip(x, totals)):
        ax.text(x_pos, -650, f'Total: {total:,}',  # Changed from -500 to -650
                ha='center', fontsize=10, fontweight='bold', color='darkblue')

    plt.tight_layout()
    plt.savefig('Figure2_Class_Distribution_Simplified.png', dpi=300, bbox_inches='tight')
    print("Figure 2 saved: Figure2_Class_Distribution_Simplified.png")
    plt.show()
    plt.close()

#### Feature Enhancement Summary

In [ ]:
def create_figure3():
    datasets = ['CIC-IoT', 'IoT-23']
    common_features = [48, 0]
    cybersecurity_features = [4, 4]
    statistical_features = [8, 8]
    pca_features = [0, 20]

    colors = ['#FF6B6B', '#FFD93D', '#6BCF7F', '#4ECDC4']

    fig, ax = plt.subplots(figsize=(10, 7))
    x = np.arange(len(datasets))
    width = 0.5

    bottom = np.zeros(len(datasets))

    feature_types = [
        ('Common Features', common_features),
        ('Cybersecurity Features', cybersecurity_features),
        ('Statistical Features', statistical_features),
        ('PCA Features', pca_features)
    ]

    for i, (label, values) in enumerate(feature_types):
        bars = ax.bar(x, values, width, label=label, bottom=bottom,
                     color=colors[i], alpha=0.85, edgecolor='black', linewidth=1.2)

        for j, (bar, val) in enumerate(zip(bars, values)):
            if val > 0:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2.,
                       bottom[j] + height/2,
                       f'{val}',
                       ha='center', va='center', fontsize=11,
                       fontweight='bold', color='white' if val > 5 else 'black')

        bottom += values

    totals = [60, 32]
    for i, (x_pos, total) in enumerate(zip(x, totals)):
        ax.text(x_pos, total + 1, f'Total: {total}',
                ha='center', va='bottom', fontsize=12, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3))

    ax.set_ylabel('Number of Features', fontsize=13, fontweight='bold')
    ax.set_xlabel('Dataset', fontsize=13, fontweight='bold')
    ax.set_title('Feature Enhancement Summary by Dataset',
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(datasets, fontsize=12)
    ax.legend(title='Feature Types', loc='upper right', framealpha=0.9, fontsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_ylim(0, max(totals) + 8)

    plt.tight_layout()
    plt.savefig('Figure3_Feature_Enhancement_Summary.png', dpi=300, bbox_inches='tight')
    print("Figure 3 saved: Figure3_Feature_Enhancement_Summary.png")
    plt.show()
    plt.close()

#### FIGURE 4: Final Transfer Learning Results - All Models

In [ ]:
def create_figure4():
    models = ['Random\nForest', 'Gradient\nBoosting', 'XGBoost', 'SVM', 'MLP']
    cic_iot_accuracy = [99.0, 98.9, 98.4, 80.0, 37.0]
    iot_23_accuracy = [50.0, 50.0, 50.0, 50.0, 50.0]

    fig, ax = plt.subplots(figsize=(12, 7))
    x = np.arange(len(models))
    width = 0.35

    cic_color = '#4ECDC4'
    iot_color = '#FF6B6B'

    bars1 = ax.bar(x - width/2, cic_iot_accuracy, width, label='CIC-IoT',
                   color=cic_color, alpha=0.85, edgecolor='black', linewidth=1.5)
    bars2 = ax.bar(x + width/2, iot_23_accuracy, width, label='IoT-23',
                   color=iot_color, alpha=0.85, edgecolor='black', linewidth=1.5)

    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}%',
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

    ax.axhline(y=99.0, color='green', linestyle='--', linewidth=2, alpha=0.5)
    ax.axhline(y=50.0, color='red', linestyle='--', linewidth=2, alpha=0.5)

    ax.set_ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Model', fontsize=14, fontweight='bold')
    ax.set_title('Final Transfer Learning Results - All Models',
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(models, fontsize=12)

    # FIXED: Move legend to better position
    ax.legend(loc='center left', fontsize=11, framealpha=0.95,
             bbox_to_anchor=(0.02, 0.5))

    ax.set_ylim(0, 105)
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.tight_layout()
    plt.savefig('Figure4_Final_Results_All_Models.png', dpi=300, bbox_inches='tight')
    print("Figure 4 saved: Figure4_Final_Results_All_Models.png")
    plt.show()
    plt.close()

#### FIGURE 6: Enhanced Transfer Learning System Architecture

In [ ]:
def create_figure6():
    from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

    fig, ax = plt.subplots(figsize=(14, 10))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')

    blue = '#ADD8E6'
    green = '#90EE90'
    yellow = '#FFD700'
    pink = '#FFB6C1'
    purple = '#DDA0DD'
    orange = '#FFA500'
    gray = '#D3D3D3'

    box_width = 3.5
    box_height = 1.2

    # 1. Raw Datasets
    raw_box = FancyBboxPatch((3.25, 8.5), box_width, box_height,
                             boxstyle="round,pad=0.1",
                             edgecolor='black', facecolor=blue, linewidth=2)
    ax.add_patch(raw_box)
    ax.text(5, 9.4, 'Raw Datasets', ha='center', va='center',
            fontsize=14, fontweight='bold')
    ax.text(5, 9.0, 'CICIoMT, CIC-IoT', ha='center', va='center', fontsize=11)
    ax.text(5, 8.7, 'IoT-23', ha='center', va='center', fontsize=11)

    # 2. Preprocessing
    prep_box = FancyBboxPatch((3.25, 6.8), box_width, box_height,
                              boxstyle="round,pad=0.1",
                              edgecolor='black', facecolor=green, linewidth=2)
    ax.add_patch(prep_box)
    ax.text(5, 7.6, 'Preprocessing', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 7.2, '& Label Mapping', ha='center', va='center', fontsize=11)

    # 3. Feature Alignment
    align_box = FancyBboxPatch((0.5, 5.1), box_width, box_height,
                               boxstyle="round,pad=0.1",
                               edgecolor='black', facecolor=yellow, linewidth=2)
    ax.add_patch(align_box)
    ax.text(2.25, 5.9, 'Enhanced Feature', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(2.25, 5.5, 'Alignment', ha='center', va='center', fontsize=11)

    # 4. Class Balancing
    balance_box = FancyBboxPatch((6, 5.1), box_width, box_height,
                                 boxstyle="round,pad=0.1",
                                 edgecolor='black', facecolor=pink, linewidth=2)
    ax.add_patch(balance_box)
    ax.text(7.75, 5.9, 'Class', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(7.75, 5.5, 'Balancing', ha='center', va='center', fontsize=11)

    # 5. Model Training
    train_box = FancyBboxPatch((3.25, 3.4), box_width, box_height,
                               boxstyle="round,pad=0.1",
                               edgecolor='black', facecolor=purple, linewidth=2)
    ax.add_patch(train_box)
    ax.text(5, 4.2, 'Model Training', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 3.8, 'XGBoost, RF, SVM, MLP, GB', ha='center', va='center', fontsize=10)

    # 6. Transfer Learning
    transfer_box = FancyBboxPatch((3.25, 1.7), box_width, box_height,
                                  boxstyle="round,pad=0.1",
                                  edgecolor='black', facecolor=orange, linewidth=2)
    ax.add_patch(transfer_box)
    ax.text(5, 2.5, 'Transfer Learning', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 2.1, 'Optimization', ha='center', va='center', fontsize=11)

    # 7. Final Results
    result_box = FancyBboxPatch((3.25, 0), box_width, box_height,
                                boxstyle="round,pad=0.1",
                                edgecolor='black', facecolor=gray, linewidth=2)
    ax.add_patch(result_box)
    ax.text(5, 0.8, 'Final Results', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 0.4, '99.0% Accuracy (CIC-IoT)', ha='center', va='center',
            fontsize=11, fontweight='bold', color='green')

    # Arrows
    arrow_props = dict(arrowstyle='->', lw=2.5, color='black')

    ax.add_patch(FancyArrowPatch((5, 8.5), (5, 8.0), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 6.8), (5, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 6.3), (2.25, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((2.25, 6.3), (2.25, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 6.3), (7.75, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((7.75, 6.3), (7.75, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((2.25, 5.1), (5, 4.6), **arrow_props))
    ax.add_patch(FancyArrowPatch((7.75, 5.1), (5, 4.6), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 3.4), (5, 2.9), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 1.7), (5, 1.2), **arrow_props))

    ax.set_title('Enhanced Transfer Learning System Architecture',
                 fontsize=18, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.savefig('Figure6_System_Architecture.png', dpi=300, bbox_inches='tight')
    print("Figure 6 saved: Figure6_System_Architecture.png")
    plt.show()
    plt.close()

#### Figure7: Transfer Learning Performance Evolution

In [ ]:
def create_figure7():
    phases = ['Initial\nMisalignment', 'Basic\nAlignment', 'Class\nBalancing',
              'Enhanced\nAlignment', 'Final\nOptimization']

    cic_iot_evolution = [45, 72, 85, 92, 99.0]
    iot_23_evolution = [35, 58, 72, 82, 50.0]

    fig, ax = plt.subplots(figsize=(12, 7))
    x = np.arange(len(phases))

    line1 = ax.plot(x, cic_iot_evolution, marker='o', markersize=10,
                    linewidth=3, color='#2E7D32', label='CIC-IoT (Successful Transfer)',
                    markerfacecolor='#4CAF50', markeredgecolor='black', markeredgewidth=2)

    line2 = ax.plot(x, iot_23_evolution, marker='s', markersize=10,
                    linewidth=3, color='#C62828', label='IoT-23 (Failed Transfer)',
                    markerfacecolor='#EF5350', markeredgecolor='black', markeredgewidth=2,
                    linestyle='--')

    for i, (val1, val2) in enumerate(zip(cic_iot_evolution, iot_23_evolution)):
        ax.text(i, val1 + 2, f'{val1:.1f}%', ha='center', va='bottom',
               fontsize=11, fontweight='bold', color='#2E7D32',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.7))
        ax.text(i, val2 - 2, f'{val2:.1f}%', ha='center', va='top',
               fontsize=11, fontweight='bold', color='#C62828',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', alpha=0.7))

    ax.axhline(y=99.0, color='green', linestyle=':', linewidth=2, alpha=0.5)
    ax.axhline(y=50.0, color='red', linestyle=':', linewidth=2, alpha=0.5)

    ax.set_ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Development Phase', fontsize=14, fontweight='bold')
    ax.set_title('Transfer Learning Performance Evolution',
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(phases, fontsize=11)
    ax.legend(loc='lower right', fontsize=12, framealpha=0.95)
    ax.set_ylim(25, 105)
    ax.grid(True, alpha=0.3, linestyle='--')

    success_note = "48 common features\nProtocol similarity\nDomain compatibility"
    ax.text(0.98, 0.82, success_note, transform=ax.transAxes,
           fontsize=9, verticalalignment='top', horizontalalignment='right',
           bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

    failure_note = "0 common features\nSemantic domain shift\nProtocol mismatch"
    ax.text(0.98, 0.35, failure_note, transform=ax.transAxes,
           fontsize=9, verticalalignment='top', horizontalalignment='right',
           bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))

    plt.tight_layout()
    plt.savefig('Figure7_Performance_Evolution.png', dpi=300, bbox_inches='tight')
    print("Figure 7 saved: Figure7_Performance_Evolution.png")
    plt.show()
    plt.close()

In [ ]:
if __name__ == "__main__":
    print("Generating thesis figures with actual experimental results...")
    print("=" * 70)

    create_figure1()
    create_figure2()
    create_figure3()
    create_figure4()
    create_figure6()  # Now with fixed imports
    create_figure7()

    print("=" * 70)
    print("All figures generated successfully!")
    print("\nGenerated files:")
    print("  1. Figure1_Training_Time_Comparison.png")
    print("  2. Figure2_Class_Distribution_Simplified.png")
    print("  3. Figure3_Feature_Enhancement_Summary.png")
    print("  4. Figure4_Final_Results_All_Models.png")
    print("  6. Figure6_System_Architecture.png")
    print("  7. Figure7_Performance_Evolution.png")
    print("\nAll figures are publication-ready at 300 DPI")

In [ ]:
"""
THESIS FIGURES - CHAPTER 5: DESIGN, IMPLEMENTATION, AND EXPERIMENTAL ANALYSIS
Complete figure generation script with proper numbering and documentation

Author: Oluwaseyi Oladejo
Date: November 2025
Thesis: Leveraging Cross-Domain Transfer Learning for Enhanced Multi-Protocol Network Intrusion Detection

FIGURE ORDER (as they appear in Chapter 5):
- Figure 1: Feature Enhancement Summary by Dataset (Section 5.2.2)
- Figure 2: Feature Alignment Before and After Enhancement (Section 5.3)
- Figure 3: Binary Class Distribution - DoS vs. Reconnaissance (Section 5.4)
- Figure 4: Model Training Time Comparison Across Datasets (Section 5.5)
- Figure 5: Final Transfer Learning Results - All Models (Section 5.6)
- Figure 6: Transfer Learning Performance Evolution (Section 5.6)
- Figure 7: Enhanced Transfer Learning System Architecture (Section 5.7)
"""

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

# Set publication-quality style
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.dpi'] = 300


# ============================================================================
# FIGURE 1: Feature Enhancement Summary by Dataset
# Location: Section 5.2.2 - Feature Engineering and Augmentation
# ============================================================================

def create_figure1_feature_enhancement():
    """
    Stacked bar chart showing feature composition breakdown:
    - CIC-IoT: 48 common + 4 cyber + 8 stat + 0 PCA = 60 total
    - IoT-23: 0 common + 4 cyber + 8 stat + 20 PCA = 32 total
    """
    print("Generating Figure 1: Feature Enhancement Summary...")

    datasets = ['CIC-IoT', 'IoT-23']
    common_features = [48, 0]
    cybersecurity_features = [4, 4]
    statistical_features = [8, 8]
    pca_features = [0, 20]

    colors = ['#FF6B6B', '#FFD93D', '#6BCF7F', '#4ECDC4']

    fig, ax = plt.subplots(figsize=(10, 7))
    x = np.arange(len(datasets))
    width = 0.5
    bottom = np.zeros(len(datasets))

    feature_types = [
        ('Common Features', common_features),
        ('Cybersecurity Features', cybersecurity_features),
        ('Statistical Features', statistical_features),
        ('PCA Features', pca_features)
    ]

    for i, (label, values) in enumerate(feature_types):
        bars = ax.bar(x, values, width, label=label, bottom=bottom,
                     color=colors[i], alpha=0.85, edgecolor='black', linewidth=1.2)

        for j, (bar, val) in enumerate(zip(bars, values)):
            if val > 0:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2.,
                       bottom[j] + height/2, f'{val}',
                       ha='center', va='center', fontsize=11,
                       fontweight='bold', color='white' if val > 5 else 'black')
        bottom += values

    totals = [60, 32]
    for i, (x_pos, total) in enumerate(zip(x, totals)):
        ax.text(x_pos, total + 1, f'Total: {total}',
                ha='center', va='bottom', fontsize=12, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3))

    ax.set_ylabel('Number of Features', fontsize=13, fontweight='bold')
    ax.set_xlabel('Dataset', fontsize=13, fontweight='bold')
    ax.set_title('Feature Enhancement Summary by Dataset',
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(datasets, fontsize=12)
    ax.legend(title='Feature Types', loc='upper right', framealpha=0.9, fontsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_ylim(0, max(totals) + 8)

    plt.tight_layout()
    plt.savefig('Figure1_Feature_Enhancement_Summary.png', dpi=300, bbox_inches='tight')
    print("  Saved: Figure1_Feature_Enhancement_Summary.png")
    plt.show()
    plt.close()


# ============================================================================
# FIGURE 2: Feature Alignment Before and After Enhancement
# Location: Section 5.3 - Feature Alignment for Transfer Learning
# ============================================================================

def create_figure2_feature_alignment():
    """
    Side-by-side comparison showing transformation:
    - BEFORE: CICIoT (44 features), IoT-23 (0 features)
    - AFTER: CICIoT (73 features), IoT-23 (29 features)
    """
    print("Generating Figure 2: Feature Alignment Before/After...")

    datasets = ['CICIoT', 'IoT-23']
    before_features = [44, 0]
    after_features = [73, 29]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    problem_color = '#FF6B6B'
    success_color = '#6BCF7F'

    # LEFT: Before Alignment
    x1 = np.arange(len(datasets))
    bars_before = ax1.bar(x1, before_features, width=0.6,
                          color=problem_color, alpha=0.85,
                          edgecolor='black', linewidth=2)

    for i, (bar, val) in enumerate(zip(bars_before, before_features)):
        ax1.text(bar.get_x() + bar.get_width()/2., val + 1,
                f'{val}', ha='center', va='bottom', fontsize=14, fontweight='bold')

    ax1.set_ylabel('Number of Features', fontsize=13, fontweight='bold')
    ax1.set_xlabel('Dataset', fontsize=13, fontweight='bold')
    ax1.set_title('Before: Feature Misalignment Problem',
                  fontsize=14, fontweight='bold', pad=15, color='darkred')
    ax1.set_xticks(x1)
    ax1.set_xticklabels(datasets, fontsize=12)
    ax1.set_ylim(0, 80)
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.text(0.5, 0.95, 'Feature overlap issue:\nIoT-23 has ZERO common features',
            transform=ax1.transAxes, ha='center', va='top',
            fontsize=10, bbox=dict(boxstyle='round,pad=0.5',
            facecolor='lightcoral', alpha=0.7))

    # RIGHT: After Alignment
    x2 = np.arange(len(datasets))
    bars_after = ax2.bar(x2, after_features, width=0.6,
                         color=success_color, alpha=0.85,
                         edgecolor='black', linewidth=2)

    for i, (bar, val) in enumerate(zip(bars_after, after_features)):
        ax2.text(bar.get_x() + bar.get_width()/2., val + 1,
                f'{val}', ha='center', va='bottom', fontsize=14, fontweight='bold')

    ax2.text(0, 73 - 10, '48 common\n+4 cyber\n+8 stat\n+13 PCA',
            ha='center', va='top', fontsize=9,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

    ax2.text(1, 29 - 5, '0 common\n+4 cyber\n+8 stat\n+17 PCA',
            ha='center', va='top', fontsize=9,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

    ax2.set_ylabel('Number of Features', fontsize=13, fontweight='bold')
    ax2.set_xlabel('Dataset', fontsize=13, fontweight='bold')
    ax2.set_title('After: Enhanced Alignment Success',
                  fontsize=14, fontweight='bold', pad=15, color='darkgreen')
    ax2.set_xticks(x2)
    ax2.set_xticklabels(datasets, fontsize=12)
    ax2.set_ylim(0, 80)
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    ax2.text(0.5, 0.95, 'Solution: Feature engineering\nenables transfer learning evaluation',
            transform=ax2.transAxes, ha='center', va='top',
            fontsize=10, bbox=dict(boxstyle='round,pad=0.5',
            facecolor='lightgreen', alpha=0.7))

    fig.suptitle('Feature Alignment Before and After Enhancement',
                 fontsize=16, fontweight='bold', y=1.02)

    plt.tight_layout()
    plt.savefig('Figure2_Feature_Alignment_BeforeAfter.png', dpi=300, bbox_inches='tight')
    print("  Saved: Figure2_Feature_Alignment_BeforeAfter.png")
    plt.show()
    plt.close()


# ============================================================================
# FIGURE 3: Binary Class Distribution - DoS vs. Reconnaissance
# Location: Section 5.4 - Addressing Class Imbalance Via Hybrid Balancing
# ============================================================================

def create_figure3_class_distribution():
    """
    Bar chart showing class distribution across source and target datasets:
    - CICIoMT: 6,999 DoS (72.3%), 2,684 Recon (27.7%)
    - CICIoT: 1,500 DoS (91.7%), 136 Recon (8.3%)
    - IoT-23: 1,500 DoS (50%), 1,500 Recon (50%)
    """
    print("Generating Figure 3: Class Distribution...")

    datasets = ['CICIOMT\n(Source)', 'CICIoT\n(Target)', 'IoT-23\n(Target)']
    dos_counts = [6999, 1500, 1500]
    recon_counts = [2684, 136, 1500]
    dos_percentages = [72.3, 91.7, 50.0]
    recon_percentages = [27.7, 8.3, 50.0]
    totals = [9683, 1636, 3000]

    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(datasets))
    width = 0.35

    dos_color = '#FF6B6B'
    recon_color = '#4ECDC4'

    bars1 = ax.bar(x - width/2, dos_counts, width, label='DoS',
                   color=dos_color, alpha=0.8, edgecolor='black', linewidth=1.2)
    bars2 = ax.bar(x + width/2, recon_counts, width, label='Reconnaissance',
                   color=recon_color, alpha=0.8, edgecolor='black', linewidth=1.2)

    for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
        height1 = bar1.get_height()
        ax.text(bar1.get_x() + bar1.get_width()/2., height1,
                f'{dos_counts[i]:,}\n({dos_percentages[i]:.1f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

        height2 = bar2.get_height()
        ax.text(bar2.get_x() + bar2.get_width()/2., height2,
                f'{recon_counts[i]:,}\n({recon_percentages[i]:.1f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

    ax.set_ylabel('Sample Count', fontsize=13, fontweight='bold')
    ax.set_xlabel('Dataset', fontsize=13, fontweight='bold')
    ax.set_title('Binary Class Distribution: DoS vs. Reconnaissance',
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(datasets, fontsize=11)
    ax.legend(loc='upper right', fontsize=11, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_ylim(-800, max(dos_counts) * 1.15)

    for i, (x_pos, total) in enumerate(zip(x, totals)):
        ax.text(x_pos, -650, f'Total: {total:,}',
                ha='center', fontsize=10, fontweight='bold', color='darkblue')

    plt.tight_layout()
    plt.savefig('Figure3_Class_Distribution.png', dpi=300, bbox_inches='tight')
    print("  Saved: Figure3_Class_Distribution.png")
    plt.show()
    plt.close()


# ============================================================================
# FIGURE 4: Model Training Time Comparison
# Location: Section 5.5 - Multi-Model Transfer Learning Implementation
# ============================================================================

def create_figure4_training_time():
    """
    Log-scale bar chart showing training times for 5 models on 2 datasets
    """
    print("Generating Figure 4: Training Time Comparison...")

    training_data = {
        'CIC-IoT': {
            'XGBoost': 1.00,
            'RandomForest': 1.70,
            'SVM': 0.57,
            'MLP': 13.84,
            'GradientBoosting': 21.98
        },
        'IoT-23': {
            'XGBoost': 0.61,
            'RandomForest': 0.93,
            'SVM': 6.82,
            'MLP': 7.68,
            'GradientBoosting': 9.68
        }
    }

    models = list(training_data['CIC-IoT'].keys())
    datasets = list(training_data.keys())

    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(datasets))
    width = 0.15

    colors = {
        'XGBoost': '#FF6B6B',
        'RandomForest': '#4ECDC4',
        'SVM': '#45B7D1',
        'MLP': '#FFA07A',
        'GradientBoosting': '#98D8C8'
    }

    for i, model in enumerate(models):
        values = [training_data[dataset][model] for dataset in datasets]
        ax.bar(x + i * width, values, width, label=model, color=colors[model])

    ax.set_ylabel('Training Time (seconds, log scale)', fontsize=13, fontweight='bold')
    ax.set_xlabel('Target Dataset', fontsize=13, fontweight='bold')
    ax.set_title('Model Training Time Comparison Across Datasets',
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x + width * 2)
    ax.set_xticklabels(datasets)
    ax.legend(title='Models', loc='upper left', framealpha=0.9)
    ax.set_yscale('log')
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.tight_layout()
    plt.savefig('Figure4_Training_Time_Comparison.png', dpi=300, bbox_inches='tight')
    print("  Saved: Figure4_Training_Time_Comparison.png")
    plt.show()
    plt.close()


# ============================================================================
# FIGURE 5: Final Transfer Learning Results - All Models
# Location: Section 5.6 - Experimental Progression (first figure)
# ============================================================================

def create_figure5_final_results():
    """
    Bar chart showing actual experimental results:
    - CIC-IoT: RF=99%, GB=98.9%, XGB=98.4%, SVM=80%, MLP=37%
    - IoT-23: All models=50%
    """
    print("Generating Figure 5: Final Results All Models...")

    models = ['Random\nForest', 'Gradient\nBoosting', 'XGBoost', 'SVM', 'MLP']
    cic_iot_accuracy = [99.0, 98.9, 98.4, 80.0, 37.0]
    iot_23_accuracy = [50.0, 50.0, 50.0, 50.0, 50.0]

    fig, ax = plt.subplots(figsize=(12, 7))
    x = np.arange(len(models))
    width = 0.35

    cic_color = '#4ECDC4'
    iot_color = '#FF6B6B'

    bars1 = ax.bar(x - width/2, cic_iot_accuracy, width, label='CIC-IoT',
                   color=cic_color, alpha=0.85, edgecolor='black', linewidth=1.5)
    bars2 = ax.bar(x + width/2, iot_23_accuracy, width, label='IoT-23',
                   color=iot_color, alpha=0.85, edgecolor='black', linewidth=1.5)

    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}%', ha='center', va='bottom',
                   fontsize=11, fontweight='bold')

    ax.axhline(y=99.0, color='green', linestyle='--', linewidth=2, alpha=0.5)
    ax.axhline(y=50.0, color='red', linestyle='--', linewidth=2, alpha=0.5)

    ax.set_ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Model', fontsize=14, fontweight='bold')
    ax.set_title('Final Transfer Learning Results - All Models',
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(models, fontsize=12)
    ax.legend(loc='center left', fontsize=11, framealpha=0.95,
             bbox_to_anchor=(0.02, 0.5))
    ax.set_ylim(0, 105)
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.tight_layout()
    plt.savefig('Figure5_Final_Results_All_Models.png', dpi=300, bbox_inches='tight')
    print("  Saved: Figure5_Final_Results_All_Models.png")
    plt.show()
    plt.close()


# ============================================================================
# FIGURE 6: Transfer Learning Performance Evolution
# Location: Section 5.6 - Experimental Progression (second figure)
# ============================================================================

def create_figure6_performance_evolution():
    """
    Line chart showing performance evolution across 5 development phases:
    - CIC-IoT: 45% → 72% → 85% → 92% → 99%
    - IoT-23: 35% → 58% → 72% → 82% → 50%
    """
    print("Generating Figure 6: Performance Evolution...")

    phases = ['Initial\nMisalignment', 'Basic\nAlignment', 'Class\nBalancing',
              'Enhanced\nAlignment', 'Final\nOptimization']

    cic_iot_evolution = [45, 72, 85, 92, 99.0]
    iot_23_evolution = [35, 58, 72, 82, 50.0]

    fig, ax = plt.subplots(figsize=(12, 7))
    x = np.arange(len(phases))

    line1 = ax.plot(x, cic_iot_evolution, marker='o', markersize=10,
                    linewidth=3, color='#2E7D32', label='CIC-IoT (Successful Transfer)',
                    markerfacecolor='#4CAF50', markeredgecolor='black', markeredgewidth=2)

    line2 = ax.plot(x, iot_23_evolution, marker='s', markersize=10,
                    linewidth=3, color='#C62828', label='IoT-23 (Failed Transfer)',
                    markerfacecolor='#EF5350', markeredgecolor='black', markeredgewidth=2,
                    linestyle='--')

    for i, (val1, val2) in enumerate(zip(cic_iot_evolution, iot_23_evolution)):
        ax.text(i, val1 + 2, f'{val1:.1f}%', ha='center', va='bottom',
               fontsize=11, fontweight='bold', color='#2E7D32',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.7))
        ax.text(i, val2 - 2, f'{val2:.1f}%', ha='center', va='top',
               fontsize=11, fontweight='bold', color='#C62828',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', alpha=0.7))

    ax.axhline(y=99.0, color='green', linestyle=':', linewidth=2, alpha=0.5)
    ax.axhline(y=50.0, color='red', linestyle=':', linewidth=2, alpha=0.5)

    ax.set_ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Development Phase', fontsize=14, fontweight='bold')
    ax.set_title('Transfer Learning Performance Evolution',
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(phases, fontsize=11)
    ax.legend(loc='lower right', fontsize=12, framealpha=0.95)
    ax.set_ylim(25, 105)
    ax.grid(True, alpha=0.3, linestyle='--')

    success_note = "48 common features\nProtocol similarity\nDomain compatibility"
    ax.text(0.98, 0.82, success_note, transform=ax.transAxes,
           fontsize=9, verticalalignment='top', horizontalalignment='right',
           bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

    failure_note = "0 common features\nSemantic domain shift\nProtocol mismatch"
    ax.text(0.98, 0.35, failure_note, transform=ax.transAxes,
           fontsize=9, verticalalignment='top', horizontalalignment='right',
           bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))

    plt.tight_layout()
    plt.savefig('Figure6_Performance_Evolution.png', dpi=300, bbox_inches='tight')
    print("  Saved: Figure6_Performance_Evolution.png")
    plt.show()
    plt.close()


# ============================================================================
# FIGURE 7: Enhanced Transfer Learning System Architecture
# Location: Section 5.7 - System Architecture Overview
# ============================================================================

def create_figure7_system_architecture():
    """
    Flowchart showing the 7-stage transfer learning pipeline
    """
    print("Generating Figure 7: System Architecture...")

    fig, ax = plt.subplots(figsize=(14, 10))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')

    blue = '#ADD8E6'
    green = '#90EE90'
    yellow = '#FFD700'
    pink = '#FFB6C1'
    purple = '#DDA0DD'
    orange = '#FFA500'
    gray = '#D3D3D3'

    box_width = 3.5
    box_height = 1.2

    # 1. Raw Datasets
    raw_box = FancyBboxPatch((3.25, 8.5), box_width, box_height,
                             boxstyle="round,pad=0.1",
                             edgecolor='black', facecolor=blue, linewidth=2)
    ax.add_patch(raw_box)
    ax.text(5, 9.4, 'Raw Datasets', ha='center', va='center',
            fontsize=14, fontweight='bold')
    ax.text(5, 9.0, 'CICIoMT, CICIoT', ha='center', va='center', fontsize=11)
    ax.text(5, 8.7, 'IoT-23', ha='center', va='center', fontsize=11)

    # 2. Preprocessing
    prep_box = FancyBboxPatch((3.25, 6.8), box_width, box_height,
                              boxstyle="round,pad=0.1",
                              edgecolor='black', facecolor=green, linewidth=2)
    ax.add_patch(prep_box)
    ax.text(5, 7.6, 'Preprocessing', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 7.2, '& Label Mapping', ha='center', va='center', fontsize=11)

    # 3. Feature Alignment
    align_box = FancyBboxPatch((0.5, 5.1), box_width, box_height,
                               boxstyle="round,pad=0.1",
                               edgecolor='black', facecolor=yellow, linewidth=2)
    ax.add_patch(align_box)
    ax.text(2.25, 5.9, 'Enhanced Feature', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(2.25, 5.5, 'Alignment', ha='center', va='center', fontsize=11)

    # 4. Class Balancing
    balance_box = FancyBboxPatch((6, 5.1), box_width, box_height,
                                 boxstyle="round,pad=0.1",
                                 edgecolor='black', facecolor=pink, linewidth=2)
    ax.add_patch(balance_box)
    ax.text(7.75, 5.9, 'Class', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(7.75, 5.5, 'Balancing', ha='center', va='center', fontsize=11)

    # 5. Model Training
    train_box = FancyBboxPatch((3.25, 3.4), box_width, box_height,
                               boxstyle="round,pad=0.1",
                               edgecolor='black', facecolor=purple, linewidth=2)
    ax.add_patch(train_box)
    ax.text(5, 4.2, 'Model Training', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 3.8, 'XGBoost, RF, SVM, MLP, GB', ha='center', va='center', fontsize=10)

    # 6. Transfer Learning
    transfer_box = FancyBboxPatch((3.25, 1.7), box_width, box_height,
                                  boxstyle="round,pad=0.1",
                                  edgecolor='black', facecolor=orange, linewidth=2)
    ax.add_patch(transfer_box)
    ax.text(5, 2.5, 'Transfer Learning', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 2.1, 'Optimization', ha='center', va='center', fontsize=11)

    # 7. Final Results
    result_box = FancyBboxPatch((3.25, 0), box_width, box_height,
                                boxstyle="round,pad=0.1",
                                edgecolor='black', facecolor=gray, linewidth=2)
    ax.add_patch(result_box)
    ax.text(5, 0.8, 'Final Results', ha='center', va='center',
            fontsize=13, fontweight='bold')
    ax.text(5, 0.4, '99.0% Accuracy (CIC-IoT)', ha='center', va='center',
            fontsize=11, fontweight='bold', color='green')

    # Arrows
    arrow_props = dict(arrowstyle='->', lw=2.5, color='black')

    ax.add_patch(FancyArrowPatch((5, 8.5), (5, 8.0), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 6.8), (5, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 6.3), (2.25, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((2.25, 6.3), (2.25, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 6.3), (7.75, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((7.75, 6.3), (7.75, 6.3), **arrow_props))
    ax.add_patch(FancyArrowPatch((2.25, 5.1), (5, 4.6), **arrow_props))
    ax.add_patch(FancyArrowPatch((7.75, 5.1), (5, 4.6), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 3.4), (5, 2.9), **arrow_props))
    ax.add_patch(FancyArrowPatch((5, 1.7), (5, 1.2), **arrow_props))

    ax.set_title('Enhanced Transfer Learning System Architecture',
                 fontsize=18, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.savefig('Figure7_System_Architecture.png', dpi=300, bbox_inches='tight')
    print("  Saved: Figure7_System_Architecture.png")
    plt.show()
    plt.close()


# ============================================================================
# MAIN EXECUTION - Generate ALL figures in correct order
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*70)
    print("GENERATING ALL THESIS FIGURES - CHAPTER 5")
    print("="*70 + "\n")

    create_figure1_feature_enhancement()
    create_figure2_feature_alignment()
    create_figure3_class_distribution()
    create_figure4_training_time()
    create_figure5_final_results()
    create_figure6_performance_evolution()
    create_figure7_system_architecture()

    print("\n" + "="*70)
    print("ALL FIGURES GENERATED SUCCESSFULLY!")
    print("="*70)
    print("\nGenerated files (in order):")
    print("  1. Figure1_Feature_Enhancement_Summary.png (Section 5.2.2)")
    print("  2. Figure2_Feature_Alignment_BeforeAfter.png (Section 5.3)")
    print("  3. Figure3_Class_Distribution.png (Section 5.4)")
    print("  4. Figure4_Training_Time_Comparison.png (Section 5.5)")
    print("  5. Figure5_Final_Results_All_Models.png (Section 5.6)")
    print("  6. Figure6_Performance_Evolution.png (Section 5.6)")
    print("  7. Figure7_System_Architecture.png (Section 5.7)")
    print("\nAll figures are publication-ready at 300 DPI")
    print("="*70 + "\n")
